In [ ]:
import os
from bindsnet.datasets import MNIST
from bindsnet.encoding import PoissonEncoder
from SNN_supervised import load_config, create_network, create_monitorings, train_network
from torchvision import transforms

In [ ]:
config = load_config("config.json")
network = create_network(config["n_neurons"], config["exc"], config["inh"], config["dt"], config["theta_plus"])
network, spikes = create_monitorings(network, config["time"])

dataset = MNIST(
    PoissonEncoder(time=config["time"], dt=config["dt"]),
    None,
    root=os.path.join("..", "..", "data", "MNIST"),
    download=True,
    transform=transforms.Compose(
        [transforms.ToTensor(), transforms.Lambda(lambda x: x * config["intensity"])]
    ),
)
train_network(network, dataset, spikes, config["n_train"], config["update_interval"], config["n_classes"], config["time"], config["n_neurons"], config["n_clamp"])